In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Dropout, BatchNormalization, Bidirectional, CuDNNLSTM, GRU, Conv2D, ConvLSTM2D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
#from keras.optimizers import Adam, SGD
import nltk
from keras.layers import Dense,Dropout
from keras.layers import LSTM,Conv1D
from keras.layers import MaxPooling1D
from keras.layers import Flatten
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from tensorflow.keras import regularizers

import re
from google.colab import drive
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
data = pd.read_csv('/content/gdrive/Shareddrives/National Undergraduate Big Data Challenge 2021/NEWSTUFF_March2022/data2022.csv')
data1 = data

data = data[['body_text_processed', 'reliability']]
data = data.astype(str)
df_full = data
df_full.drop(df_full.tail(1).index,inplace=True) # drop last n rows

In [ ]:
data1.head()

,Unnamed: 0,news_id,url,publisher,publish_date,author,title,image,body_text,news_guard_score,mbfc_level,political_bias,country,reliability,combined,body_text_processed
0,0,0,https://www.nytimes.com/article/what-is-corona...,The New York Times,2020-01-21,"['Knvul Sheikh', 'Roni Caryn Rabin']",The Coronavirus: What Scientists Have Learned ...,https://static01.nyt.com/images/2020/03/12/sci...,\nA novel respiratory virus that originated in...,100.0,High,Left,USA,1,The Coronavirus: What Scientists Have Learned ...,coronavirus scientist learned far. novel respi...
1,1,1,https://www.npr.org/2020/01/22/798392172/chine...,National Public Radio (NPR),2020-01-22,['Emily Feng'],Chinese Health Officials: More Die From Newly ...,https://media.npr.org/include/images/facebook-...,Chinese Health Officials: More Die From Newly ...,100.0,Very high,Center,USA,1,Chinese Health Officials: More Die From Newly ...,chinese health official die newly identified c...
2,2,2,https://www.theverge.com/2020/1/23/21078457/co...,The Verge,2020-01-23,['Nicole Wetsman'],Everything you need to know about the coronavi...,https://cdn.vox-cdn.com/thumbor/a9_Oz7cvSBKyal...,Public health experts around the globe are scr...,100.0,High,Left-center,USA,1,Everything you need to know about the coronavi...,everything need know coronavirus. public healt...
3,3,3,https://www.worldhealth.net/news/novel-coronav...,WorldHealth.Net,2020-01-24,[],Novel Coronavirus Cases Confirmed To Be Spread...,https://www.worldhealth.net/media/original_ima...,The first two coronavirus cases in Europe have...,30.0,Low,NaN,USA,0,Novel Coronavirus Cases Confirmed To Be Spread...,novel coronavirus case confirmed spreading. fi...
4,4,4,https://www.theverge.com/2020/1/24/21080845/co...,The Verge,2020-01-24,"['Nicole Wetsman', 'Zoe Schiffer', 'Jay Peters...",Coronavirus disrupts the world: updates on the...,https://cdn.vox-cdn.com/thumbor/t2gt1SmEni4Mcr...,"A new coronavirus appeared in Wuhan, China, at...",100.0,High,Left-center,USA,1,Coronavirus disrupts the world: updates on the...,coronavirus disrupts world update pandemic. ne...


In [ ]:
max_fatures = 20000 
tokenizer = Tokenizer(num_words=max_fatures, split=' ', oov_token='<unw>')
tokenizer.fit_on_texts(df_full['body_text_processed'].values)
X = tokenizer.texts_to_sequences(df_full['body_text_processed'].values)
X = pad_sequences(X, padding = 'post', maxlen = 100)

In [ ]:
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

In [ ]:
from numpy import array
from numpy import asarray
from numpy import zeros

embeddings_dictionary = dict()
glove_file = open('/content/gdrive/Shareddrives/National Undergraduate Big Data Challenge 2021/glove.6B.100d.txt', encoding="utf8")

In [ ]:
for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary [word] = vector_dimensions

glove_file.close()

In [ ]:
embedding_matrix = zeros((vocab_size, 100))
for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [ ]:
Y = (df_full['reliability']).values
Y = Y.astype(int)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.10, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

In [ ]:
#BiLSTM

model = Sequential()
model.add(Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=X.shape[1], trainable=True))
model.add(SpatialDropout1D(0.25))
model.add(Bidirectional(CuDNNLSTM(50)))
#model.add(layer = Dropout(rate = 0.4))
# model.add(Bidirectional(CuDNNLSTM(50)))
# model.add(layer = Dropout(rate = 0.4))
#model.add(layer = BatchNormalization())
#model.add(layer = Dense(units = 100,  activation = 'relu'))
model.add(layer = BatchNormalization())
model.add(layer = Dropout(rate = 0.4))
model.add(Dense(1,activation='sigmoid'))
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss = 'binary_crossentropy', optimizer=optimizer,metrics = ['accuracy'])
print(model.summary())


In [ ]:
batch_size = 32
history = model.fit(X_train, Y_train, epochs = 10, batch_size=batch_size, validation_split=0.2)
#+20

In [ ]:
#CNN

model1 = Sequential()
model1.add(Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=X.shape[1], trainable=True))
model1.add(SpatialDropout1D(0.2))

model1.add(Conv1D(filters=100,
                        kernel_size=4,
                        activation='relu', padding="same", kernel_regularizer = tf.keras.regularizers.l2(l2=0.001)))
model1.add(layer = BatchNormalization())
model1.add(MaxPooling1D(pool_size=2, padding = 'same'))
model1.add(layer = Dropout(rate = 0.2))
model1.add(Flatten())
model1.add(layer = Dense(units = 100,  activation = 'relu'))
model1.add(layer = Dropout(rate = 0.2))
model1.add(Dense(1,activation='sigmoid'))
optimizer = Adam(learning_rate=0.001)
model1.compile(loss = 'binary_crossentropy', optimizer=optimizer,metrics = ['accuracy'])
print(model1.summary())
#, dropout=0.4, recurrent_dropout=0.4)

In [ ]:
batch_size = 2 
history1 = model1.fit(X_train, Y_train, epochs = 15, batch_size=batch_size, validation_split=0.2)

In [ ]:
#LSTM

model2 = Sequential()
model2.add(Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=X.shape[1], trainable=True))
model2.add(SpatialDropout1D(0.25))
model2.add(LSTM(100))

model2.add(layer = Dropout(rate = 0.5))
# model.add(Bidirectional(CuDNNLSTM(50)))
# model.add(layer = Dropout(rate = 0.4))
#model.add(layer = BatchNormalization())
#model2.add(layer = Dense(units = 100,  activation = 'relu'))
#model.add(layer = BatchNormalization())
#model.add(layer = Dropout(rate = 0.1))
model2.add(Dense(1,activation='sigmoid'))
optimizer = Adam(learning_rate=0.005)
model2.compile(loss = 'binary_crossentropy', optimizer=optimizer,metrics = ['accuracy'])
print(model2.summary())
#, dropout=0.4, recurrent_dropout=0.4)

In [ ]:
batch_size = 128 
history2 = model2.fit(X_train, Y_train, epochs = 10, batch_size=batch_size, validation_split=0.2)

In [ ]:
#BiGRU
model3 = Sequential()
model3.add(Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=X.shape[1], trainable=True))
model3.add(SpatialDropout1D(0.3))
model3.add(Bidirectional(GRU(50)))
model3.add(layer = Dropout(rate = 0.3))
#model.add(layer = Dense(units = 100,  activation = 'relu'))
#model.add(layer = BatchNormalization())
#model.add(layer = Dropout(rate = 0.1))
model3.add(Dense(1,activation='sigmoid'))
optimizer = Adam(learning_rate=0.001)
model3.compile(loss = 'binary_crossentropy', optimizer=optimizer,metrics = ['accuracy'])
print(model3.summary())
#, dropout=0.4, recurrent_dropout=0.4)

In [ ]:
batch_size = 128
history3 = model3.fit(X_train, Y_train, epochs = 10, batch_size=batch_size, validation_split=0.2)

In [ ]:
#GRU
model4 = Sequential()
model4.add(Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=X.shape[1], trainable=True))
model4.add(SpatialDropout1D(0.3))
model4.add((GRU(100)))
model4.add(layer = Dropout(rate = 0.5))
# model.add(Bidirectional(CuDNNLSTM(50)))
# model.add(layer = Dropout(rate = 0.4))
#model.add(layer = BatchNormalization())
#model.add(layer = Dense(units = 100,  activation = 'relu'))
#model.add(layer = BatchNormalization())
#model.add(layer = Dropout(rate = 0.1))
model4.add(Dense(1,activation='sigmoid'))
optimizer = Adam(learning_rate=0.005)
model4.compile(loss = 'binary_crossentropy', optimizer=optimizer,metrics = ['accuracy'])
print(model4.summary())
#, dropout=0.4, recurrent_dropout=0.4)

In [ ]:
batch_size = 128 
history4 = model4.fit(X_train, Y_train, epochs = 10, batch_size=batch_size, validation_split=0.2)

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
y_pred_keras = model.predict(X_test)
fpr_keras, tpr_keras, thresholds_keras = roc_curve(Y_test, y_pred_keras)
auc_keras = auc(fpr_keras, tpr_keras)

In [ ]:
from sklearn.metrics import classification_report

y_pred_bool = y_pred_keras.round()

print(classification_report(Y_test, y_pred_bool))

In [ ]:
from sklearn.metrics import precision_recall_fscore_support as score

precision, recall, fscore, support = score(Y_test, y_pred_bool, average = 'binary')

print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))
print('support: {}'.format(support))

In [ ]:
y_pred_keras1 = model1.predict(X_test)
fpr_keras1, tpr_keras1, thresholds_keras1 = roc_curve(Y_test, y_pred_keras1)
auc_keras1 = auc(fpr_keras1, tpr_keras1)

In [ ]:
y_pred_bool = y_pred_keras1.round()

print(classification_report(Y_test, y_pred_bool))

In [ ]:
precision, recall, fscore, support = score(Y_test, y_pred_bool, average = 'binary')

print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))
print('support: {}'.format(support))

In [ ]:
y_pred_keras2 = model2.predict(X_test)
fpr_keras2, tpr_keras2, thresholds_keras2 = roc_curve(Y_test, y_pred_keras2)
auc_keras2 = auc(fpr_keras2, tpr_keras2)

In [ ]:
y_pred_bool = y_pred_keras2.round()

print(classification_report(Y_test, y_pred_bool))

In [ ]:
precision, recall, fscore, support = score(Y_test, y_pred_bool, average = 'binary')

print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))
print('support: {}'.format(support))

In [ ]:
y_pred_keras3 = model3.predict(X_test)
fpr_keras3, tpr_keras3, thresholds_keras3 = roc_curve(Y_test, y_pred_keras3)
auc_keras3 = auc(fpr_keras3, tpr_keras3)

In [ ]:
y_pred_bool = y_pred_keras3.round()

print(classification_report(Y_test, y_pred_bool))

In [ ]:
precision, recall, fscore, support = score(Y_test, y_pred_bool, average = 'binary')

print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))
print('support: {}'.format(support))

In [ ]:
y_pred_keras4 = model4.predict(X_test)
fpr_keras4, tpr_keras4, thresholds_keras4 = roc_curve(Y_test, y_pred_keras4)
auc_keras4 = auc(fpr_keras4, tpr_keras4)

In [ ]:
y_pred_bool = y_pred_keras4.round()

print(classification_report(Y_test, y_pred_bool))

In [ ]:
precision, recall, fscore, support = score(Y_test, y_pred_bool, average = 'binary')

print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))
print('support: {}'.format(support))

In [ ]:
model_results = pd.DataFrame()
model_results['test'] = Y_test
model_results['0'] = y_pred_keras
model_results['1'] = y_pred_keras1
model_results['2'] = y_pred_keras2
model_results['3'] = y_pred_keras3
model_results['4'] = y_pred_keras4

In [ ]:
model_results

In [ ]:
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
Naive = naive_bayes.MultinomialNB()
Naive.fit(X_train, Y_train)
# predict the labels on validation dataset
#predictions_NB = Naive.predict(X_test1)
predictions_NB = Naive.predict_proba(X_test)
predictions_NB = predictions_NB[:,1]
# Use accuracy_score function to get the accuracy
#print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, Y_test)*100)
fpr_keras_NB, tpr_keras_NB, thresholds_keras_NB = roc_curve(Y_test, predictions_NB)
auc_keras_NB = auc(fpr_keras_NB, tpr_keras_NB)

In [ ]:
from sklearn.metrics import classification_report
y_pred_bool = predictions_NB.round()

print(classification_report(Y_test, y_pred_bool))

In [ ]:
precision, recall, fscore, support = score(Y_test, y_pred_bool, average = 'binary')

print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))
print('support: {}'.format(support))

In [ ]:
predictions_NB.shape

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
KNN = KNeighborsClassifier()
KNN.fit(X_train, Y_train)
# predict the labels on validation dataset
predictions_KNN = KNN.predict_proba(X_test)
predictions_KNN = predictions_KNN[:,1]
# Use accuracy_score function to get the accuracy
#print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, Y_test)*100)
fpr_keras_KNN, tpr_keras_KNN, thresholds_keras_KNN = roc_curve(Y_test, predictions_KNN)
auc_keras_KNN = auc(fpr_keras_KNN, tpr_keras_KNN)

In [ ]:
y_pred_bool = predictions_KNN.round()

print(classification_report(Y_test, y_pred_bool))

In [ ]:
precision, recall, fscore, support = score(Y_test, y_pred_bool, average = 'binary')

print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))
print('support: {}'.format(support))

In [ ]:
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression(max_iter = 4000,class_weight='balanced')
LR.fit(X_train, Y_train)
# predict the labels on validation dataset
predictions_LR = LR.predict_proba(X_test)
predictions_LR = predictions_LR[:,1]
# Use accuracy_score function to get the accuracy
#print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, Y_test)*100)
fpr_keras_LR, tpr_keras_LR, thresholds_keras_LR = roc_curve(Y_test, predictions_LR)
auc_keras_LR = auc(fpr_keras_LR, tpr_keras_LR)

In [ ]:
predictions_LR

In [ ]:
y_pred_bool = predictions_LR.round()

print(classification_report(Y_test, y_pred_bool))

In [ ]:
precision, recall, fscore, support = score(Y_test, y_pred_bool, average = 'binary')

print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))
print('support: {}'.format(support))

In [ ]:
# SVM = svm.SVC(kernel='linear', probability = True)
# SVM.fit(X_train,Y_train)
# # predict the labels on validation dataset
# predictions_SVM = SVM.predict_proba(X_test)
# predictions_SVM = predictions_SVM[:,0]
# # Use accuracy_score function to get the accuracy
# #print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)
# fpr_keras_SVM, tpr_keras_SVM, thresholds_keras_SVM = roc_curve(Y_test, predictions_SVM)
# auc_keras_SVM = auc(fpr_keras_SVM, tpr_keras_SVM)

In [ ]:
# predictions_SVM

In [ ]:
# y_pred_bool = predictions_SVM.round()

# print(classification_report(Y_test_SVM1, y_pred_bool))

In [ ]:
from sklearn.metrics import roc_curve
y_pred_keras = model.predict(X_test)
fpr_keras, tpr_keras, thresholds_keras = roc_curve(Y_test, y_pred_keras)

In [ ]:
(X_test).shape

(203, 100)

In [ ]:
from sklearn.metrics import auc
auc_keras = auc(fpr_keras, tpr_keras)

In [ ]:
merged_df = pd.read_csv('/content/gdrive/Shareddrives/National Undergraduate Big Data Challenge 2021/NEWSTUFF_March2022/data2022_features.csv')

In [ ]:
X = merged_df.drop(merged_df.columns[[0,1,2,3]], axis=1) 
Y = (merged_df['reliability']).values
Y = Y.astype(int)
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.10, random_state = 42)

print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

In [ ]:
max_fatures = 20000 #unique words, set threshold on if it appears >3 times in dataset
tokenizer = Tokenizer(num_words=max_fatures, split=' ', oov_token='<unw>')
tokenizer.fit_on_texts(merged_df['body_text_processed'].values)
X_train_text = tokenizer.texts_to_sequences(X_train['body_text_processed'].values)
X_train_text = pad_sequences(X_train_text, padding = 'post', maxlen = 3500) #max words in article, set to largest word count 
X_train_features = X_train.drop(X_train.columns[0], axis=1) 
X_test_text = tokenizer.texts_to_sequences(X_test['body_text_processed'].values)
X_test_text = pad_sequences(X_test_text, padding = 'post', maxlen = 3500) #max words in article, set to largest word count 
X_test_features = X_test.drop(X_test.columns[0], axis=1) 

In [ ]:
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

In [ ]:
from numpy import array
from numpy import asarray
from numpy import zeros

embeddings_dictionary = dict()
glove_file = open('/content/gdrive/Shareddrives/National Undergraduate Big Data Challenge 2021/glove.6B.100d.txt', encoding="utf8")

In [ ]:
for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary [word] = vector_dimensions

glove_file.close()

In [ ]:
embedding_matrix = zeros((vocab_size, 100))
for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [ ]:
#GRU
from keras.layers import Dense, Embedding, LSTM, Bidirectional, Input, concatenate, GRU
model8 = Sequential()
model8.add(Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=X_train_text.shape[1], trainable=True))
model8.add(SpatialDropout1D(0.5))
model8.add(Bidirectional(GRU(50)))
#model4.add(layer = Dropout(rate = 0.5))
model8.add(Dense(1,activation='sigmoid'))
optimizer = Adam(learning_rate=0.001)
model8.compile(loss = 'binary_crossentropy', optimizer=optimizer,metrics = ['accuracy'])
print(model8.summary())
#, dropout=0.4, recurrent_dropout=0.4)

In [ ]:
H = model8.fit(x=[X_train_text], y=Y_train, epochs = 10, batch_size=64, validation_split=0.2,verbose=1)

In [ ]:
X_full = tokenizer.texts_to_sequences(X['body_text_processed'].values)
X_full = pad_sequences(X_full, padding = 'post', maxlen = 3500) #max words in article, set to largest word count 
X_full_features = X.drop(X.columns[0], axis=1) 
preds = model8.predict([X_full])
X_full_features['pred'] = preds

In [ ]:
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from matplotlib import pyplot

X_train, X_test, y_train, y_test = train_test_split(X_full_features, Y, test_size=0.2, random_state=42)
eval_set = [(X_train, y_train), (X_test, y_test)]

model9 = XGBClassifier(silent=False, 
                      scale_pos_weight=1,
                      learning_rate=0.01,  
                      colsample_bytree = 0.6,
                      subsample = 0.6,
                      objective='binary:logistic', 
                      n_estimators=300, 
                      reg_alpha = 0.9,
                      reg_lambda = 10,
                      max_depth=3, 
                      gamma=10, 
                      random_state = 42)

In [ ]:
model9.fit(X_train, y_train, eval_metric=["error", "logloss",'auc'], eval_set=eval_set, verbose=True)

In [ ]:
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from matplotlib import pyplot

# make predictions for test data
y_pred = model9.predict(X_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
# retrieve performance metrics
results = model9.evals_result()
epochs = len(results['validation_0']['error'])
x_axis = range(0, epochs)
# plot log loss
fig, ax = pyplot.subplots()
ax.plot(x_axis, results['validation_0']['logloss'], label='Train')
ax.plot(x_axis, results['validation_1']['logloss'], label='Test')
ax.legend()
pyplot.ylabel('Log Loss')
pyplot.title('XGBoost Log Loss')
pyplot.show()
# plot classification error
fig, ax = pyplot.subplots()
ax.plot(x_axis, results['validation_0']['error'], label='Train')
ax.plot(x_axis, results['validation_1']['error'], label='Test')
ax.legend()
pyplot.ylabel('Classification Error')
pyplot.title('XGBoost Classification Error')
pyplot.show()

In [ ]:
from sklearn.metrics import precision_recall_fscore_support as score

precision, recall, fscore, support = score(y_test, y_pred, average = 'binary')

print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))
print('support: {}'.format(support))

In [ ]:
preds_test = model8.predict(X_test_text)
X_test_features['pred'] = preds_test

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
y_pred_keras = model9.predict_proba(X_test_features)
fpr_keras9, tpr_keras9, thresholds_keras9 = roc_curve(Y_test, y_pred_keras[:,1])
auc_keras9 = auc(fpr_keras9, tpr_keras9)

In [ ]:
!cd '/content/gdrive/Shareddrives/National Undergraduate Big Data Challenge 2021'
import matplotlib.pyplot as plt

plt.plot(fpr_keras_NB, tpr_keras_NB, label='NB (area = {:.3f})'.format(auc_keras_NB))
plt.plot(fpr_keras_KNN, tpr_keras_KNN, label='KNN (area = {:.3f})'.format(auc_keras_KNN))
# plt.plot(fpr_keras_SVM, tpr_keras_SVM, label='SVM (area = {:.3f})'.format(auc_keras_SVM))
plt.plot(fpr_keras_LR, tpr_keras_LR, label='LR (area = {:.3f})'.format(auc_keras_LR))
plt.plot(fpr_keras, tpr_keras, label='BiLSTM (area = {:.3f})'.format(auc_keras))
plt.plot(fpr_keras1, tpr_keras1, label='CNN (area = {:.3f})'.format(auc_keras1))
plt.plot(fpr_keras2, tpr_keras2, label='LSTM (area = {:.3f})'.format(auc_keras2))
plt.plot(fpr_keras3, tpr_keras3, label='BiGRU (area = {:.3f})'.format(auc_keras3))
plt.plot(fpr_keras4, tpr_keras4, label='GRU (area = {:.3f})'.format(auc_keras4))

plt.plot(fpr_keras9, tpr_keras9, label='New model (area = {:.3f})'.format(auc_keras9)) #need to import model from other ipynb file


plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best', prop={'size': 8})


plt.savefig('/content/gdrive/Shareddrives/National Undergraduate Big Data Challenge 2021/DL_AUC.pdf')  

In [ ]:
def plot_confusion_matrix(cm,
                          target_names,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True):
    """
    given a sklearn confusion matrix (cm), make a nice plot

    Arguments
    ---------
    cm:           confusion matrix from sklearn.metrics.confusion_matrix

    target_names: given classification classes such as [0, 1, 2]
                  the class names, for example: ['high', 'medium', 'low']

    title:        the text to display at the top of the matrix

    cmap:         the gradient of the values displayed from matplotlib.pyplot.cm
                  see http://matplotlib.org/examples/color/colormaps_reference.html
                  plt.get_cmap('jet') or plt.cm.Blues

    normalize:    If False, plot the raw numbers
                  If True, plot the proportions

    Usage
    -----
    plot_confusion_matrix(cm           = cm,                  # confusion matrix created by
                                                              # sklearn.metrics.confusion_matrix
                          normalize    = True,                # show proportions
                          target_names = y_labels_vals,       # list of names of the classes
                          title        = best_estimator_name) # title of graph

    Citiation
    ---------
    http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html

    """
    import matplotlib.pyplot as plt
    import numpy as np
    import itertools

    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.tight_layout()
    plt.ylabel('True label')
    #plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.show()

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 16}

plt.rc('font', **font)

plot_confusion_matrix(cm           = np.array(confusion_matrix(Y_test, y_pred_keras[:,1].round())), 
                      normalize    = False,
                      target_names = ['Reliable article', 'Unreliable article'],
                      title        = " ",
                      cmap = 'GnBu')

In [ ]:
merged_df = pd.read_csv('/content/gdrive/Shareddrives/National Undergraduate Big Data Challenge 2021/NEWSTUFF_March2022/data2022_features.csv')

In [ ]:
merged_df

In [ ]:
merged_df_rel = merged_df.loc[merged_df['reliability'] == 0]

In [ ]:
merged_df_rel

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

model10 = XGBClassifier(silent=False, 
                      scale_pos_weight=1,
                      learning_rate=0.01,  
                      colsample_bytree = 0.6,
                      subsample = 0.6,
                      objective='binary:logistic', 
                      n_estimators=300, 
                      reg_alpha = 0.9,
                      reg_lambda = 10,
                      max_depth=3, 
                      gamma=10, 
                      random_state = 42)

kfold = KFold(n_splits=10)
results = cross_val_score(model10, X_full_features, Y, cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

In [ ]:
from sklearn.metrics import recall_score
from sklearn.metrics import make_scorer
specificity = make_scorer(recall_score, pos_label=0)
results = model_selection.cross_validate(model10, X_full_features, Y, cv=10, scoring = {
    'accuracy': make_scorer(accuracy_score),
    'sensitivity': make_scorer(recall_score),
    'specificity': make_scorer(recall_score,pos_label=0)
})

In [ ]:
sensitivity = make_scorer(recall_score, pos_label=1)
print(cross_val_score(model10, X_full_features, Y, cv=10, sensitivity))

In [ ]:
results['test_accuracy'].mean()

In [ ]:
results['test_specificity'].mean()

In [ ]:
results['test_sensitivity'].mean()